***IMPORTS Y ACCESO A DATOS DE CSV BICIS***

In [119]:
#Importación de paquetes y lectura de csv:
from sklearn.linear_model import Ridge
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
import pandas as pd
import numpy as np
import random
from sklearn import model_selection
from scipy.spatial.distance import euclidean
from scipy.stats import spearmanr
import xgboost as xgb
from xgboost import DMatrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import copy

random.seed(27)
dataBicis = pd.read_csv('hour.csv')
dataBicis = dataBicis.drop("dteday", axis=1)
dataBicis = dataBicis.drop("instant", axis=1)
atributosBicis = dataBicis.loc[:, 'season':'registered']
objetivoBicis = dataBicis['cnt']

(atributos_entrenamientoBicis, atributos_pruebaBicis,
 objetivo_entrenamientoBicis, objetivo_pruebaBicis) = model_selection.train_test_split(
    atributosBicis, objetivoBicis, test_size=.33)

atributos_entrenamientoBicis = atributos_entrenamientoBicis.values
objetivo_entrenamientoBicis = objetivo_entrenamientoBicis.values
atributos_pruebaBicis = atributos_pruebaBicis.values
objetivo_pruebaBicis= objetivo_pruebaBicis.values

rangosBicis = {} #Diccionario con los valores máximo y mínimo de cada atributo

for columna in dataBicis.columns:
    valor_min = dataBicis[columna].min()
    valor_max = dataBicis[columna].max()
    rangosBicis[columna] = {'min': valor_min, 'max': valor_max}

***MÉTODO LIME***

In [120]:
def metodo_lime(N,x,k,model,rangos):
    random.seed(1)
    Xprima=[]
    R=[]
    W=[]
    Yprima= []
    for i in range (0,N):
        atributosAPerturbar=seleccionar_atributos_aleatorios(x,k)
        xprima = perturbar_muestra(x,atributosAPerturbar,rangos)
        w = distancia_entre_muestras(x,xprima)
        r=atributosAPerturbar
        Xprima.append(xprima)
        R.append(r)
        W.append(w)
        yprima = prediccion_de_modelo(xprima,model)
        Yprima.append(yprima)
    W=np.array(W)
    W = W.reshape(W.shape[0])
    G = entrenar_modelo_ridge(R,Yprima,W)
    return G.coef_

#Se seleccional los k atributos aleatorios a modificar

def seleccionar_atributos_aleatorios(dataframe, k):
    columnas = dataframe.columns.tolist()
    seleccionados = random.sample(range(len(columnas)), k)
    lista = [0] * len(columnas)
    for i in seleccionados:
        lista[i] = 1
    #print(lista)
    return lista

#Se perturba la muestra original de forma que los atributos seleccionados pueden variar en un rango en función del valor máximo y mínimo de dicho atributo
def perturbar_muestra(x, atributosAPerturbar,rangos):
    xprima = x.copy()
    for i in range(0,len(atributosAPerturbar)):
        if atributosAPerturbar[i] == 1:
            rango_min = xprima[i] - rangos[list(rangos.keys())[i]]['min']
            rango_max = rangos[list(rangos.keys())[i]]['max'] - xprima[i]
            xprima[i] = random.uniform(rango_min, rango_max)
    return xprima

# Se calcula la distancia coseno entre la muestra original y la perturbada
def distancia_entre_muestras(x,xprima):
    return cosine_distances(x.values, xprima.values)

#Se predice el resultado de la muestra con el modelo elegido
def prediccion_de_modelo(xprima,model):
    if not isinstance(model, (RandomForestRegressor, RandomForestClassifier)):
        xprima = DMatrix(xprima)
    y_pred = model.predict(xprima)
    return y_pred


#Se entrena el modelo de regresión Ridge con las muestras perturbadas y los pesos calculados
def entrenar_modelo_ridge(R, Yprima, W):
    modelo_entrenado = Ridge(alpha=1.0) 
    modelo_entrenado.fit(R, Yprima, sample_weight=W)
    return modelo_entrenado

***MODELOS BICIS***

In [121]:
#RandomForest
#Creamos el modelo RandomForest (En este caso es Regressor ya que se trata de una tarea de regresión)
modelRandomForestBicis = RandomForestRegressor(n_estimators=100)
#Entrenamos el modelo con los atributos y objetivo de entrenamiento
modelRandomForestBicis.fit(atributos_entrenamientoBicis, objetivo_entrenamientoBicis)

#XGBoost
# Creamos el objeto DMatrix para XGBoost
dtrainBicis = xgb.DMatrix(atributos_entrenamientoBicis, label=objetivo_entrenamientoBicis)
dtestBicis = xgb.DMatrix(atributos_pruebaBicis)
dtestBicis2 = xgb.DMatrix(atributos_pruebaBicis)
# Definimos los parámetros del modelo XGBoost
paramsBicis = {
    'objective': 'reg:squarederror',  # Objetivo para regresión
    'eval_metric': 'rmse'  # Métrica de evaluación, error cuadrático medio (RMSE)
}

# Entrenamos el modelo XGBoost
modelXGBoostBicis = xgb.train(paramsBicis, dtrainBicis)

In [122]:
clon_model_XGboost_bicis = copy.deepcopy(modelXGBoostBicis)
prediccionesBicisXG = clon_model_XGboost_bicis.predict(dtestBicis)
explicacionesBicisXG = []
atributosAcambiar = 5
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaBicis).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, clon_model_XGboost_bicis,rangosBicis)
    explicacionesBicisXG.append(explicacion)
print(explicacionesBicisXG)

[array([[ 1.16736397e+01,  2.27378306e+00,  4.92358546e-03,
         1.43418724e-03,  1.09121459e-03,  1.04354078e+01,
        -8.83573629e+00, -1.60459509e+00, -5.78499838e+00,
         5.11012073e+00,  2.27269185e+00, -5.10915890e+00,
         1.85731549e-03, -1.04404608e+01]]), array([[-25.025366  , -32.83068902,  -8.30018538, -23.09190515,
         -1.05192665, -82.90937014,  59.37368805,  31.83586748,
         -1.71006263, -14.14234272, -31.77876237,  13.70821473,
         24.09548563,  91.82735419]]), array([[ 9.88079329e-01, -4.74774023e+00, -6.04372766e-03,
        -1.32820123e-02, -7.29877310e-04, -5.24502737e+00,
         1.24154294e+00,  4.00952815e+00,  3.23825249e+00,
        -2.52240340e+00, -4.74701036e+00,  2.52387982e+00,
         2.76768463e-02,  5.25327739e+00]]), array([[  3.56560351,  -8.06081965,  -0.31835527,  -0.70867269,
         -0.06047734, -11.17936693,   5.87191384,   5.62580836,
          0.76399562,   0.89392479,  -8.00034231,  -0.94015254,
          1.03

In [123]:
prediccionesBicisRF = modelRandomForestBicis.predict(atributos_pruebaBicis)
explicacionesBicisRF = []
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaBicis).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, modelRandomForestBicis,rangosBicis)
    explicacionesBicisRF.append(explicacion)
print(explicacionesBicisRF)

[array([[ 1.06900576e+01,  2.67641289e+00,  4.44467911e-03,
         9.01023205e-04,  9.33163687e-04,  1.15770749e+01,
        -8.76527491e+00, -2.81624463e+00, -4.46598306e+00,
         4.59934939e+00,  2.67547973e+00, -4.59850068e+00,
         2.95389465e-03, -1.15816040e+01]]), array([[-27.46079059, -33.18304128,  -8.43497578, -22.65265999,
         -1.06198165, -84.05985372,  61.48649525,  31.00833425,
          0.64732516, -14.23575102, -32.12105963,  13.80538593,
         23.13612703,  93.12644605]]), array([[ 1.31727337e+00, -4.96921503e+00, -6.00092202e-03,
        -1.29239996e-02, -7.17554922e-04, -4.33166025e+00,
         7.96536519e-01,  3.54112466e+00,  4.26554394e+00,
        -2.85940898e+00, -4.96849747e+00,  2.86086860e+00,
         2.72387779e-02,  4.33983835e+00]]), array([[  5.81910692, -10.46691615,  -0.34392447,  -0.79238979,
         -0.06344013, -10.24319473,   5.41655753,   5.17056168,
          4.17117347,   0.22367234, -10.40347602,  -0.27197766,
          1.18

***VALIDACIÓN DE MODELO***

In [124]:
def evaluate_model(model, Xentrenamiento, yentrenamiento, Xtest, ytest):
    # Validación cruzada
    cv_scores = cross_val_score(model, Xentrenamiento, yentrenamiento, cv=2)
    mean_cv_score = np.mean(cv_scores)
    
    print("Puntuación media de validación cruzada:", mean_cv_score)
    
    #.score
    score = model.score(Xtest, ytest)
    print("Puntuación en el conjunto de prueba:", score)

In [125]:
evaluate_model(modelRandomForestBicis, atributos_entrenamientoBicis, objetivo_entrenamientoBicis, atributos_pruebaBicis, objetivo_pruebaBicis)

Puntuación media de validación cruzada: 0.9995192181741472
Puntuación en el conjunto de prueba: 0.9996862714053326


******

In [126]:


nuevos_ejemplos2 = pd.DataFrame(atributos_entrenamientoBicis).iloc[3]
dataframe = pd.DataFrame([nuevos_ejemplos2])
print(dataframe)


Iterac=10
atributosAcambiar=5

nuevos_ejemplos = pd.DataFrame([[1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0,1,7]])
#print("Prediccion RF",prediccion_de_modelo(dataframe,modelRandomForestBicis))
#print("Prediccion XG",prediccion_de_modelo(dataframe,modelXGBoostBicis))
#print("Método LIME con Random Forest: ",metodo_lime(Iterac,dataframe,atributosAcambiar,modelRandomForestBicis,rangosBicis))

#print("Método LIME con XGBoost: ",metodo_lime(Iterac,dataframe,atributosAcambiar,modelXGBoostBicis,rangosBicis))

    0    1    2     3    4    5    6    7     8       9     10      11    12  \
3  2.0  1.0  4.0  13.0  0.0  1.0  1.0  2.0  0.48  0.4697  0.51  0.0896  48.0   

      13  
3  199.0  


******

***

***ACCESO A DATOS DE CSV PEN***

In [127]:

# Cargar el archivo CSV
pen = pd.read_csv('pendigits.csv', header=0, names=['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10','a11','a12','a13','a14','a15','a16',"class-objective"])

atributosPen = pen.loc[:, 'a1':'a16']
objetivoPen = pen['class-objective']

rangosPen = {} #Diccionario con los valores máximo y mínimo de cada atributo

for columna in pen.columns:
    valor_min = pen[columna].min()
    valor_max = pen[columna].max()
    rangosPen[columna] = {'min': valor_min, 'max': valor_max}
    


In [128]:
(atributos_entrenamientoPen, atributos_pruebaPen,
 objetivo_entrenamientoPen, objetivo_pruebaPen) = model_selection.train_test_split(
    atributosPen, objetivoPen, test_size=.33,stratify=objetivoPen)

atributos_entrenamientoPen = atributos_entrenamientoPen.values
objetivo_entrenamientoPen = objetivo_entrenamientoPen.values
atributos_pruebaPen = atributos_pruebaPen.values
objetivo_pruebaPen= objetivo_pruebaPen.values

print(atributos_entrenamientoPen)

[[ 29  78  52 ...   0   0  20]
 [  0  26  15 ...  29  25   0]
 [  0  88  40 ...  44 100  56]
 ...
 [ 37  78   8 ...  86 100 100]
 [ 59  87  23 ...  85  43 100]
 [  0  79  40 ...   0   4   3]]


***MODELOS PEN***

In [129]:

modelRandomForestPen = RandomForestClassifier(n_estimators=100)
#Entrenamos el modelo con los atributos y objetivo de entrenamiento
modelRandomForestPen.fit(atributos_entrenamientoPen, objetivo_entrenamientoPen)

#XGBoost
# Creamos el objeto DMatrix para XGBoost
dtrainPen = xgb.DMatrix(atributos_entrenamientoPen, label=objetivo_entrenamientoPen)
dtestPen = xgb.DMatrix(atributos_pruebaPen)
dtestPen2 = xgb.DMatrix(atributos_pruebaPen)

# Definimos los parámetros del modelo XGBoost
paramsPen = {
    'objective': 'multi:softmax',  # Función objetivo para clasificación multiclase
    'num_class': 10,       # Número de clases en tu problema
    'eval_metric': 'merror' 
}

# Entrenamos el modelo XGBoost
modelXGBoostPen = xgb.train(paramsPen, dtrainPen)

In [130]:
clon_model_XGboost_pen = copy.deepcopy(modelXGBoostPen)
prediccionesPenXG = clon_model_XGboost_pen.predict(dtestPen)
explicacionesPenXG = []
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaPen).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, clon_model_XGboost_pen, rangosPen)
    explicacionesPenXG.append(explicacion)
print(explicacionesPenXG)

[array([[-0.03050945,  0.04591732,  0.04134104,  0.23102226,  0.05336712,
        -0.15289715,  0.04337529, -0.21221595,  0.09155531,  0.02507726,
         0.00543219, -0.20537696, -0.10624681,  0.10337329,  0.10231679,
        -0.03553156]]), array([[ 0.14729061, -0.36464147, -0.04007909,  0.05293941, -0.2669081 ,
        -0.04498637,  0.38470124, -0.02175948,  0.38792605, -0.05929629,
        -0.08799432,  0.02154443, -0.40680195, -0.01549379,  0.02708985,
         0.28646928]]), array([[-0.26280724, -0.05700818, -0.10064719, -0.25005022, -0.08270323,
        -0.06623721,  0.43118422, -0.01087697,  0.0464132 ,  0.4236861 ,
        -0.16087886, -0.2950008 , -0.06824968,  0.13758374, -0.14326549,
         0.45885779]]), array([[-0.05713656, -0.1001486 ,  0.13634889,  0.16739242,  0.03875556,
        -0.46783762,  0.29864792, -0.19864458,  0.53954656, -0.10519339,
         0.16232995, -0.09409483, -0.47073795,  0.09533629, -0.00832788,
         0.06376383]]), array([[-0.1110468 , -0.208

In [131]:
prediccionesPenRF = modelRandomForestPen.predict(atributos_pruebaPen)
explicacionesPenRF = []
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaPen).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, modelRandomForestPen, rangosPen)
    explicacionesPenRF.append(explicacion)
print(explicacionesPenRF)

[array([[-0.08186361,  0.02019642, -0.31663462,  0.06784668, -0.00634144,
        -0.28473594,  0.09801418, -0.03071405, -0.14455347,  0.05229008,
         0.02957352,  0.04335292,  0.13752907,  0.22583982,  0.16513116,
         0.02506927]]), array([[-0.09901256, -0.02739568, -0.01439323, -0.05450916, -0.05037611,
        -0.03997517, -0.0727462 , -0.02607176,  0.05752642, -0.07261266,
         0.17162521,  0.11859409, -0.0798335 ,  0.08981148,  0.16689288,
        -0.06752405]]), array([[-0.55602442, -0.09093111, -0.13095786, -0.4676566 , -0.16396569,
        -0.15900242,  0.17941152, -0.02689306,  0.1305355 ,  0.17307481,
         0.38294961,  0.05518877, -0.22208857,  0.33514989,  0.38875678,
         0.17245285]]), array([[ 0.14348648, -0.09814809,  0.13421595,  0.15612672,  0.04882249,
        -0.27296673,  0.34368789, -0.0065822 ,  0.35230214, -0.05445938,
        -0.0890271 , -0.13218389, -0.57568455,  0.19634994,  0.03375401,
        -0.1796937 ]]), array([[-0.04466403,  0.070

In [132]:
nuevos_ejemplos2 = pd.DataFrame(atributos_entrenamientoPen).iloc[2]
dataframe = pd.DataFrame([nuevos_ejemplos2])
print(dataframe)
Iterac=10
atributosAcambiar=5


#print("Prediccion RF",prediccion_de_modelo(dataframe,modelRandomForestPen))
#print("Prediccion XG",prediccion_de_modelo(dataframe,modelXGBoostPen))
#print("Método LIME con Random Forest: ",metodo_lime(Iterac,dataframe,atributosAcambiar,modelRandomForestPen,rangosPen))
#print("Método LIME con XGBoost: ",metodo_lime(Iterac,dataframe,atributosAcambiar,modelXGBoostPen,rangosPen))

   0   1   2    3   4   5   6   7   8   9   10  11  12  13   14  15
2   0  88  40  100  68  82  56  40  39   0  29  27  58  44  100  56


***VALIDACIÓN***

In [133]:
evaluate_model(modelRandomForestPen, atributos_entrenamientoPen, objetivo_entrenamientoPen, atributos_pruebaPen, objetivo_pruebaPen)

Puntuación media de validación cruzada: 0.9846613545816734
Puntuación en el conjunto de prueba: 0.9846340477153255


In [134]:
def eliminar_irrelevantes_RF(pesos, k, muestra, modelo):
    array = np.array(pesos).flatten()
    pesos_abs = np.abs(array)
    indices = np.argsort(pesos_abs)[:k]
    copia = muestra.copy()
    for i in range(len(muestra)):
        if i in indices:
            copia[i] = 0
    muestra_2d = copia.reshape(1, -1)
    prediccion_nueva = modelo.predict(muestra_2d)
    return prediccion_nueva


def eliminar_irrelevantes_XG(pesos, k, muestra, modelo):
    array = np.array(pesos).flatten()
    pesos_abs = np.abs(array)
    indices = np.argsort(pesos_abs)[:k]
    
    data = muestra.get_data()
    data_copy = data.copy()
    label = muestra.get_label()
    weight = muestra.get_weight()
    base_margin = muestra.get_base_margin()

    # Copiar los valores y crear una nueva muestra
    
    for i in indices:
        data_copy[0,i]=0
    copia = xgb.DMatrix(data_copy, label=label, weight=weight, base_margin=base_margin)
    prediccion_nueva = modelo.predict(copia)
    return prediccion_nueva




In [135]:
predicciones_RF_Bicis = prediccionesBicisRF[:256]
objetivos_bicis = objetivo_pruebaBicis[:256]
explicaciones_RF_Bicis = explicacionesBicisRF

predicciones_XG_Bicis = prediccionesBicisXG[:256]
explicaciones_XG_Bicis = explicacionesBicisXG

predicciones_RF_Pen = prediccionesPenRF[:256]
objetivos_pen = objetivo_pruebaPen[:256]
explicaciones_RF_Pen = explicacionesPenRF

predicciones_XG_Pen = prediccionesPenXG[:256]
explicaciones_XG_Pen = explicacionesPenXG

In [136]:
dtestBicis = xgb.DMatrix(atributos_pruebaBicis)
dtestPen = xgb.DMatrix(atributos_pruebaPen)
predicciones_alteradas_RF_Bicis=[]
predicciones_alteradas_RF_Pen=[]
predicciones_alteradas_XG_Bicis=[]
predicciones_alteradas_XG_Pen=[]


for i in range(256):
    pred = eliminar_irrelevantes_RF(explicaciones_RF_Bicis[i],5,atributos_pruebaBicis[i],modelRandomForestBicis)
    predicciones_alteradas_RF_Bicis.append(pred)
    pred = eliminar_irrelevantes_RF(explicaciones_RF_Pen[i],5,atributos_pruebaPen[i],modelRandomForestPen)
    predicciones_alteradas_RF_Pen.append(pred)
    pred = eliminar_irrelevantes_XG(explicaciones_XG_Pen[i],5,dtestPen2.slice([i]),modelXGBoostPen)
    predicciones_alteradas_XG_Pen.append(pred)
    pred = eliminar_irrelevantes_XG(explicaciones_XG_Bicis[i],5,dtestBicis2.slice([i]),modelXGBoostBicis)
    predicciones_alteradas_XG_Bicis.append(pred)
print(len(predicciones_alteradas_XG_Bicis))
print(len(predicciones_alteradas_XG_Pen))

256
256


***MÉTRICAS***

In [137]:
# Identidad

def identidad(i, muestra, lista_explicaciones):
    e = lista_explicaciones[i]
    return np.linalg.norm(muestra - muestra) != 0 or np.linalg.norm(e - e) == 0

# Separabilidad

def separabilidad(i,j,muestra_i,muestra_j, lista_explicaciones):
    e1 = lista_explicaciones[i]
    e2 = lista_explicaciones[j]
    return np.linalg.norm(muestra_i - muestra_j) == 0 or np.linalg.norm(e1 - e2) > 0 

# Estabilidad

def estabilidad(i, muestra, lista_muestras, lista_explicaciones):
    e = lista_explicaciones[i]
    d_muestras = np.array([np.linalg.norm(muestra - x) for x in lista_muestras])
    d_explicaciones = np.array([np.linalg.norm(e - x) for x in lista_explicaciones])
    return spearmanr(d_muestras, d_explicaciones)[0] > 0

# Selectividad

def selectividad(i):
    return null

# Coherencia
def coherencia(i, lista_predicciones, lista_objetivo,predicciones_actualizadas):
    error_prediccion_original = lista_predicciones[i] - lista_objetivo[i]
    error_prediccion_nuevo = predicciones_actualizadas[i] - lista_objetivo[i]
    return np.abs(error_prediccion_original - error_prediccion_nuevo)

# Completitud

def completitud(i, lista_predicciones, lista_objetivo,predicciones_actualizadas):
    error_prediccion_original = lista_predicciones[i] - lista_objetivo[i]
    error_prediccion_nuevo = predicciones_actualizadas[i] - lista_objetivo[i]
    return error_prediccion_nuevo / error_prediccion_original

# Congruencia
def congruencia(lista_predicciones, lista_objetivo,predicciones_actualizadas):
    sumatorio_coherencia = 0
    sumatorio_desviacion = 0
    for i in range(len(lista_predicciones)):
        alpha = coherencia(i,lista_predicciones, lista_objetivo,predicciones_actualizadas)
        sumatorio_coherencia += alpha
    coherencia_promedio = sumatorio_coherencia / len(lista_predicciones)
    for i in range(len(lista_predicciones)):
        alpha = coherencia(i,lista_predicciones, lista_objetivo,predicciones_actualizadas)
        sumatorio_desviacion += np.power(alpha-coherencia_promedio,2)
    arg = sumatorio_desviacion / len(lista_predicciones)
    return np.sqrt(arg)

In [138]:
# Bicis Random Forest

# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i,atributos_pruebaBicis[i],explicaciones_RF_Bicis)
print("\nIdentidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, i+1,atributos_pruebaBicis[i],atributos_pruebaBicis[i+1],explicaciones_RF_Bicis)
print("\nSeparabilidad:", res_separabilidad)

# Estabilidad

res_estabilidad = True
for i in range(0, 256):
    res_estabilidad &= estabilidad(i,atributos_pruebaBicis[i],atributos_pruebaBicis[:256],explicaciones_RF_Bicis)
print("\nEstabilidad:", res_estabilidad)

# Selectividad

res_selectividad = True

# Coherencia

res_coherencia = 0
for i in range(0, 256):
    res_coherencia += coherencia(i,predicciones_RF_Bicis,objetivos_bicis,predicciones_alteradas_RF_Bicis)
print("\nCoherencia:", res_coherencia/256)
    
# Completitud

res_completitud = 0
for i in range(0, 256):
    res_completitud += completitud(i,predicciones_RF_Bicis,objetivos_bicis,predicciones_alteradas_RF_Bicis)
print("\nCompletitud:", res_completitud/256)

# Congruencia

print ("\nCongruencia:", congruencia(predicciones_RF_Bicis, objetivos_bicis,predicciones_alteradas_RF_Bicis))


Identidad: True

Separabilidad: True

Estabilidad: False

Coherencia: [18.26550781]

Completitud: [nan]

Congruencia: [25.88825029]


<ipython-input-137-31ef2e0c730b>:38: RuntimeWarning: invalid value encountered in divide
  return error_prediccion_nuevo / error_prediccion_original
<ipython-input-137-31ef2e0c730b>:38: RuntimeWarning: divide by zero encountered in divide
  return error_prediccion_nuevo / error_prediccion_original


In [139]:
# Bicis XGBoost

# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i,dtestBicis.slice([i]).get_data(),explicaciones_XG_Bicis)
print("\nIdentidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, i+1,dtestBicis.slice([i]).get_data(),dtestBicis.slice([i+1]).get_data,explicaciones_XG_Bicis)
print("\nSeparabilidad:", res_separabilidad)

# Estabilidad

res_estabilidad = True
lista_bicis = [x for x in dtestBicis[:256]]
for i in range(0, 256):
    res_estabilidad &= estabilidad(i,dtestBicis.slice([i]).get_data(),lista_bicis,explicaciones_XG_Bicis)
print("\nEstabilidad:", res_estabilidad)

# Selectividad

res_selectividad = True

# Coherencia

res_coherencia = 0
for i in range(0, 256):
    res_coherencia += coherencia(i,predicciones_XG_Bicis,objetivos_bicis,predicciones_alteradas_XG_Bicis)
print("\nCoherencia:", res_coherencia/256)

# Completitud

res_completitud = 0
for i in range(0, 256):
    res_completitud += completitud(i,predicciones_XG_Bicis,objetivos_bicis,predicciones_alteradas_XG_Bicis)
print("\nCompletitud:", res_completitud/256)

# Congruencia

print ("\nCongruencia:", congruencia(predicciones_XG_Bicis, objetivos_bicis,predicciones_alteradas_XG_Bicis))

ValueError: dimension mismatch

In [140]:
# Pen Random Forest

# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i,atributos_pruebaPen[i],explicaciones_RF_Pen)
print("\nIdentidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, i+1,atributos_pruebaPen[i],atributos_pruebaPen[i+1],explicaciones_RF_Pen)
print("\nSeparabilidad:", res_separabilidad)

# Estabilidad

res_estabilidad = True
for i in range(0, 256):
    res_estabilidad &= estabilidad(i,atributos_pruebaPen[i],atributos_pruebaPen[:256],explicaciones_RF_Pen)
print("\nEstabilidad:", res_estabilidad)

# Selectividad

res_selectividad = True

# Coherencia

res_coherencia = 0
for i in range(0, 256):
    res_coherencia += coherencia(i,predicciones_RF_Pen,objetivos_pen,predicciones_alteradas_RF_Pen)
print("\nCoherencia:", res_coherencia/256)
    
# Completitud

res_completitud = 0
for i in range(0, 256):
    res_completitud += completitud(i,predicciones_RF_Pen,objetivos_pen,predicciones_alteradas_RF_Pen)
print("\nCompletitud:", res_completitud/256)

# Congruencia

print ("\nCongruencia:", congruencia(predicciones_RF_Pen, objetivos_pen,predicciones_alteradas_RF_Pen))


Identidad: True

Separabilidad: False

Estabilidad: False

Coherencia: [1.328125]

Completitud: [nan]

Congruencia: [2.19413058]


<ipython-input-137-31ef2e0c730b>:38: RuntimeWarning: divide by zero encountered in divide
  return error_prediccion_nuevo / error_prediccion_original
<ipython-input-137-31ef2e0c730b>:38: RuntimeWarning: invalid value encountered in divide
  return error_prediccion_nuevo / error_prediccion_original


In [142]:
# Pen XGBoost

# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i,dtestPen.slice([i]).get_data(),explicaciones_RF_Bicis)
print("\nIdentidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, i+1,dtestPen.slice([i]).get_data(),dtestPen.slice([i+1]).get_data(),explicaciones_RF_Bicis)
print("\nSeparabilidad:", res_separabilidad)

# Estabilidad

res_estabilidad = True
lista_pen = [x for x in dtestPen[:256]]
for i in range(0, 256):
    res_estabilidad &= estabilidad(i,dtestPen.slice([i]).get_data(),lista_pen,explicaciones_RF_Bicis)
print("\nEstabilidad:", res_estabilidad)

# Selectividad

res_selectividad = True

# Coherencia

res_coherencia = 0
for i in range(0, 256):
    res_coherencia += coherencia(i,predicciones_XG_Pen,objetivos_pen,predicciones_alteradas_XG_Pen)
print("\nCoherencia:", res_coherencia/256)
    
# Completitud

res_completitud = 0
for i in range(0, 256):
    res_completitud += completitud(i,predicciones_XG_Pen,objetivos_pen,predicciones_alteradas_XG_Pen)
print("\nCompletitud", res_completitud/256)

# Congruencia

print ("\nCongruencia:", congruencia(predicciones_XG_Pen, objetivos_pen,predicciones_alteradas_XG_Pen))

ValueError: dimension mismatch